In [2]:
import pandas as pd
import numpy as np
#from bs4 import BeautifulSoup #to beautify xml output
from lxml import etree as ET #read and write xml files
pd.options.display.max_columns=50

import os
from os import listdir
from os.path import isfile, join

import time

import geopandas as gp

In [12]:
taz=gp.read_file('../SF-CHAMP Outputs/TAZ2454_clean.shp')

In [13]:
taz=taz.to_crs('EPSG:26910')
taz.crs

<Projected CRS: EPSG:26910>
Name: NAD83 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - 126°W to 120°W and NAD83 by country
- bounds: (-126.0, 30.54, -119.99, 81.8)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [14]:
taz=taz[['TAZ','X_COORD','Y_COORD','SQ_MILE']]
taz.head()

,TAZ,X_COORD,Y_COORD,SQ_MILE
0,2005,6.062583e+06,2.142762e+06,2.62483
1,1999,6.056350e+06,2.135772e+06,0.25465
2,1998,6.054933e+06,2.133579e+06,0.41488
3,2000,6.054171e+06,2.135816e+06,0.26872
4,2001,6.051967e+06,2.136249e+06,0.22250


In [15]:
taz['area']=taz['SQ_MILE']*2.59e6
taz=taz.rename(columns={'TAZ':'taz',
                       'X_COORD':'coord-x',
                       'Y_COORD':'coord-y'})
taz=taz.drop(columns=['SQ_MILE'])
taz.head()

,taz,coord-x,coord-y,area
0,2005,6.062583e+06,2.142762e+06,6798309.7
1,1999,6.056350e+06,2.135772e+06,659543.5
2,1998,6.054933e+06,2.133579e+06,1074539.2
3,2000,6.054171e+06,2.135816e+06,695984.8
4,2001,6.051967e+06,2.136249e+06,576275.0


In [19]:
taz.to_csv('../SF_CHAMP_Converted/taz-centers.csv.gz',index=False,compression='gzip')

In [20]:
proto_parking=pd.read_csv('taz-parking.txt')
proto_parking.head()

,taz,parkingType,pricingModel,chargingType,numStalls,feeInCents,ReservedFor
0,1,Public,Block,NoCharger,167,932.83514,Any
1,1,Public,Block,NoCharger,5,0.00000,Any
2,1,Public,Block,NoCharger,1478,284.01965,Any
3,1,Public,Block,NoCharger,213,0.00000,Any
4,1,Residential,Block,NoCharger,16,0.00000,Any


In [21]:
proto_parking.to_csv('../SF_CHAMP_Converted/taz-parking.csv.gz',index=False,compression='gzip')

In [23]:
toll_fac=pd.read_csv('../SF-CHAMP Outputs/toll_facility.csv')
toll_fac

,LINK_ID,TOLLAM_DA,TOLLAM_SR2,TOLLAM_SR3,TOLLPM_DA,TOLLPM_SR2,TOLLPM_SR3,TOLLEA_DA,TOLLEA_SR2,TOLLEA_SR3,TOLLMD_DA,TOLLMD_SR2,TOLLMD_SR3,TOLLEV_DA,TOLLEV_SR2,TOLLEV_SR3
0,27511,277.55,138.775,138.775,277.55,138.775,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55
1,27948,277.55,138.775,138.775,277.55,138.775,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55
2,33901,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55
3,36257,333.06,333.060,138.775,333.06,333.060,138.775,222.04,222.04,222.04,222.04,222.04,222.04,222.04,222.04,222.04
4,37032,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,37160,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,38667,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,38677,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55
8,39625,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55
9,39630,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55


In [24]:
toll_fac['toll']=5
toll_fac['timeRange']="[:]"

toll_fac=toll_fac.rename(columns={'LINK_ID':'linkId'})
toll_fac[['linkId','toll','timeRange']].to_csv('../SF_CHAMP_Converted/toll.csv.gz',compression='gzip')

In [25]:
toll_fac

,linkId,TOLLAM_DA,TOLLAM_SR2,TOLLAM_SR3,TOLLPM_DA,TOLLPM_SR2,TOLLPM_SR3,TOLLEA_DA,TOLLEA_SR2,TOLLEA_SR3,TOLLMD_DA,TOLLMD_SR2,TOLLMD_SR3,TOLLEV_DA,TOLLEV_SR2,TOLLEV_SR3,toll,timeRange
0,27511,277.55,138.775,138.775,277.55,138.775,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
1,27948,277.55,138.775,138.775,277.55,138.775,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
2,33901,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
3,36257,333.06,333.060,138.775,333.06,333.060,138.775,222.04,222.04,222.04,222.04,222.04,222.04,222.04,222.04,222.04,5,[:]
4,37032,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,[:]
5,37160,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,[:]
6,38667,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,[:]
7,38677,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
8,39625,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
9,39630,277.55,277.550,138.775,277.55,277.550,138.775,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,277.55,5,[:]
